In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim

https://www.cdc.gov/nchs/pressroom/sosmap/firearm_mortality/firearm.htm
https://open.cdc.gov/apis.html

In [2]:
#Gather data from API
data_url='data.cdc.gov'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='489q-934x'    # The data set at the API endpoint (311 data in this case)
app_token='##################'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint


# Set the timeout to 60 seconds    
client.timeout = 60

# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, cause_of_death="Firearm-related injury")

# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)

#filter to crude values
df = df[df['rate_type'] == 'Crude']

#drop cols
df.drop(['rate_type',
         'cause_of_death',
         'unit',
         'rate_sex_female', 
         'rate_sex_male',
         'rate_age_1_4', 
         'rate_age_5_14', 
         'rate_age_15_24',
         'rate_age_25_34', 
         'rate_age_35_44', 
         'rate_age_45_54', 
         'rate_age_55_64',
         'rate_65_74', 
         'rate_age_75_84', 
         'rate_age_85_plus'
        ], axis=1, inplace=True)

# drop last 2 rows
df.drop(df.tail(2).index,inplace=True) 


In [3]:
df.head(5)

,year_and_quarter,time_period,rate_overall,rate_alaska,rate_alabama,rate_arkansas,rate_arizona,rate_california,rate_colorado,rate_connecticut,...,rate_south_dakota,rate_tennessee,rate_texas,rate_utah,rate_virginia,rate_vermont,rate_washington,rate_wisconsin,rate_west_virginia,rate_wyoming
24,2020 Q1,12 months ending with quarter,12.3,23.4,21.6,19.5,15.7,7.6,14.8,5.5,...,12,19,12.9,12.1,12.7,11.2,11.1,10.7,17.7,25
25,2020 Q1,3-month period,12.3,22.4,22.9,20.5,16.3,7.9,14.7,5.9,...,8.5,18.3,13.1,11.8,13.9,10.9,11.1,10.6,19.2,27.5
26,2020 Q2,12 months ending with quarter,12.6,23.3,22.5,21.7,15.2,7.7,15.2,5.4,...,11.9,19,13.3,12.8,12.5,11.5,11.2,10.9,17.5,26
27,2020 Q2,3-month period,13.3,20.8,23.4,26.1,15.9,8.1,16.8,4.6,...,11.7,19.1,14.1,13.9,12,10.9,11.5,11,16.8,23.4
28,2020 Q3,12 months ending with quarter,13.1,25.3,22.8,21.2,16.2,8,16,5.7,...,12.3,20,13.6,12.9,13.1,11.5,11.3,11.8,16.3,28.7


In [4]:
df.columns

Index(['year_and_quarter', 'time_period', 'rate_overall', 'rate_alaska',
       'rate_alabama', 'rate_arkansas', 'rate_arizona', 'rate_california',
       'rate_colorado', 'rate_connecticut', 'rate_district_of_columbia',
       'rate_delaware', 'rate_florida', 'rate_georgia', 'rate_hawaii',
       'rate_iowa', 'rate_idaho', 'rate_illinois', 'rate_indiana',
       'rate_kansas', 'rate_kentucky', 'rate_louisiana', 'rate_massachusetts',
       'rate_maryland', 'rate_maine', 'rate_michigan', 'rate_minnesota',
       'rate_missouri', 'rate_mississippi', 'rate_montana',
       'rate_north_carolina', 'rate_north_dakota', 'rate_nebraska',
       'rate_new_hampshire', 'rate_new_jersey', 'rate_new_mexico',
       'rate_nevada', 'rate_new_york', 'rate_ohio', 'rate_oklahoma',
       'rate_oregon', 'rate_pennsylvania', 'rate_rhode_island',
       'rate_south_carolina', 'rate_south_dakota', 'rate_tennessee',
       'rate_texas', 'rate_utah', 'rate_virginia', 'rate_vermont',
       'rate_washington',

### US State Mapping

In [5]:
state_name_map = {
        'alaska': 'Alaska',
        'alabama': 'Alabama',
        'arkansas': 'Arkansas',
        'arizona': 'Arizona',
        'california': 'California',
        'colorado': 'Colorado',
        'connecticut': 'Connecticut',
        'district_of_columbia': 'District Of Columbia',
        'delaware': 'Delaware',
        'florida': 'Florida',
        'georgia': 'Georgia',
        'hawaii': 'Hawaii',
        'iowa': 'Iowa',
        'idaho': 'Idaho',
        'illinois': 'Illinois',
        'indiana': 'Indiana',
        'kansas': 'Kansas',
        'kentucky': 'Kentucky',
        'louisiana': 'Louisiana',
        'massachusetts': 'Massachusetts',
        'maryland': 'Maryland',
        'maine': 'Maine',
        'michigan': 'Michigan',
        'minnesota': 'Minnesota',
        'missouri': 'Missouri',
        'mississippi': 'Mississippi',
        'montana': 'Montana',
        'north_carolina': 'North Carolina',
        'north_dakota': 'North Dakota',
        'nebraska': 'Nebraska',
        'new_hampshire': 'New Hampshire',
        'new_jersey': 'New Jersey',
        'new_mexico': 'New Mexico',
        'nevada': 'Nevada',
        'new_york': 'New York',
        'ohio': 'Ohio',
        'oklahoma': 'Oklahoma',
        'oregon': 'Oregon',
        'pennsylvania': 'Pennsylvania',
        'rhode_island': 'Rhode Island',
        'south_carolina': 'South Carolina',
        'south_dakota': 'South Dakota',
        'tennessee': 'Tennessee',
        'texas': 'Texas',
        'utah': 'Utah',
        'virginia': 'Virginia',
        'vermont': 'Vermont',
        'washington': 'Washington',
        'wisconsin': 'Wisconsin',
        'west_virginia': 'West Virginia',
        'wyoming': 'Wyoming'
}

### Split dataframe into 3 month period and 12 month periods

In [6]:
#Quarterly values
df_qtr = df[df['time_period'] == '3-month period']

#convert to long format (states)
df_qtr = pd.melt(df_qtr, id_vars='year_and_quarter', value_vars=['rate_alaska',
       'rate_alabama', 'rate_arkansas', 'rate_arizona', 'rate_california',
       'rate_colorado', 'rate_connecticut', 'rate_district_of_columbia',
       'rate_delaware', 'rate_florida', 'rate_georgia', 'rate_hawaii',
       'rate_iowa', 'rate_idaho', 'rate_illinois', 'rate_indiana',
       'rate_kansas', 'rate_kentucky', 'rate_louisiana', 'rate_massachusetts',
       'rate_maryland', 'rate_maine', 'rate_michigan', 'rate_minnesota',
       'rate_missouri', 'rate_mississippi', 'rate_montana',
       'rate_north_carolina', 'rate_north_dakota', 'rate_nebraska',
       'rate_new_hampshire', 'rate_new_jersey', 'rate_new_mexico',
       'rate_nevada', 'rate_new_york', 'rate_ohio', 'rate_oklahoma',
       'rate_oregon', 'rate_pennsylvania', 'rate_rhode_island',
       'rate_south_carolina', 'rate_south_dakota', 'rate_tennessee',
       'rate_texas', 'rate_utah', 'rate_virginia', 'rate_vermont',
       'rate_washington', 'rate_wisconsin', 'rate_west_virginia',
       'rate_wyoming'], var_name='state')

#clean state name
df_qtr['state'] = df_qtr['state'].str[5:]
df_qtr = df_qtr.replace({"state": state_name_map})

#Convert 'year_and_quarter' column to datetime
df_qtr['qtr'] = pd.to_datetime(df_qtr['year_and_quarter'].str.replace(' ', '')) + pd.offsets.QuarterEnd(0)


df_qtr.head(5)

,year_and_quarter,state,value,qtr
0,2020 Q1,Alaska,22.4,2020-03-31
1,2020 Q2,Alaska,20.8,2020-06-30
2,2020 Q3,Alaska,26.8,2020-09-30
3,2020 Q4,Alaska,25.7,2020-12-31
4,2021 Q1,Alaska,19.1,2021-03-31


In [7]:
#Yearly values
df_yr = df[df['time_period'] == '12 months ending with quarter']

#convert to long format (states)
df_yr = pd.melt(df_yr, id_vars='year_and_quarter', value_vars=['rate_alaska',
       'rate_alabama', 'rate_arkansas', 'rate_arizona', 'rate_california',
       'rate_colorado', 'rate_connecticut', 'rate_district_of_columbia',
       'rate_delaware', 'rate_florida', 'rate_georgia', 'rate_hawaii',
       'rate_iowa', 'rate_idaho', 'rate_illinois', 'rate_indiana',
       'rate_kansas', 'rate_kentucky', 'rate_louisiana', 'rate_massachusetts',
       'rate_maryland', 'rate_maine', 'rate_michigan', 'rate_minnesota',
       'rate_missouri', 'rate_mississippi', 'rate_montana',
       'rate_north_carolina', 'rate_north_dakota', 'rate_nebraska',
       'rate_new_hampshire', 'rate_new_jersey', 'rate_new_mexico',
       'rate_nevada', 'rate_new_york', 'rate_ohio', 'rate_oklahoma',
       'rate_oregon', 'rate_pennsylvania', 'rate_rhode_island',
       'rate_south_carolina', 'rate_south_dakota', 'rate_tennessee',
       'rate_texas', 'rate_utah', 'rate_virginia', 'rate_vermont',
       'rate_washington', 'rate_wisconsin', 'rate_west_virginia',
       'rate_wyoming'], var_name='state')

#clean state name
df_yr['state'] = df_yr['state'].str[5:]
df_yr = df_yr.replace({"state": state_name_map})

#Convert 'year_and_quarter' column to datetime
df_yr['qtr'] = pd.to_datetime(df_yr['year_and_quarter'].str.replace(' ', '')) + pd.offsets.QuarterEnd(0)

# Save the data frame to a CSV file
#df_yr.to_csv("data/gun_control_results_by_yr.csv")

df_yr.head(5)

,year_and_quarter,state,value,qtr
0,2020 Q1,Alaska,23.4,2020-03-31
1,2020 Q2,Alaska,23.3,2020-06-30
2,2020 Q3,Alaska,25.3,2020-09-30
3,2020 Q4,Alaska,23.9,2020-12-31
4,2021 Q1,Alaska,23.1,2021-03-31


## Get gun score cards from website

In [8]:
#Get gun score card data
gun_sc = pd.read_html('https://giffords.org/lawcenter/resources/scorecard/')
gun_sc_df = gun_sc[0]
gun_sc_df.drop(gun_sc_df.tail(1).index,inplace=True) # drop last n rows

#Redefine grade values to be binned into a 5 scale
replace_values = {'A-' : 'A', 
                  'B+' : 'B', 
                  'B' : 'B',
                  'B-': 'C',
                  'C+': 'C',
                  'C': 'D',
                  'C-': 'D',
                  'D+': 'E',
                  'F': 'E'}                                                                                          
gun_sc_df = gun_sc_df.replace({"Grade": replace_values})


# Save the data frame to a CSV file
#gun_sc_df.to_csv("data/gun_sc_results.csv")

In [9]:
#Adding lat and long values for state
#initialize API
geolocator = Nominatim(user_agent='MyApp', timeout=10)

#convert each string state name to lat/lon
lat_arr = []
long_arr = []
for state in gun_sc_df['State']:
    lat_long_coords = geolocator.geocode(state)
    lat_arr.append(lat_long_coords.raw['lat'])
    long_arr.append(lat_long_coords.raw['lon'])
    
#insert lat/lon values to df
gun_sc_df.insert(loc=1, column='long', value=long_arr)
gun_sc_df.insert(loc=1, column='lat', value=lat_arr)


In [10]:
gun_sc_df.head(5)

,Gun Law Strength (Ranked),lat,long,State,Grade,Gun Death Rate (Ranked),Gun Death Rate (per 100K)
0,38,33.2588817,-86.8295337,Alabama,E,4,26.4
1,41,64.4459613,-149.680909,Alaska,E,6,25.2
2,42,34.395342,-111.763275,Arizona,E,17,18.3
3,50,35.2048883,-92.4479108,Arkansas,E,8,23.3
4,1,36.7014631,-118.755997,California,A,43,9.0


## Join dataframes

In [11]:
#Left join score card to qtr data
df_qtr = pd.merge(df_qtr, gun_sc_df, how="left", left_on='state', right_on='State')

# Save the data frame to a CSV file
df_qtr.to_csv("data/gun_control_results_by_qtr.csv")

df_qtr.head(5)

,year_and_quarter,state,value,qtr,Gun Law Strength (Ranked),lat,long,State,Grade,Gun Death Rate (Ranked),Gun Death Rate (per 100K)
0,2020 Q1,Alaska,22.4,2020-03-31,41,64.4459613,-149.680909,Alaska,E,6,25.2
1,2020 Q2,Alaska,20.8,2020-06-30,41,64.4459613,-149.680909,Alaska,E,6,25.2
2,2020 Q3,Alaska,26.8,2020-09-30,41,64.4459613,-149.680909,Alaska,E,6,25.2
3,2020 Q4,Alaska,25.7,2020-12-31,41,64.4459613,-149.680909,Alaska,E,6,25.2
4,2021 Q1,Alaska,19.1,2021-03-31,41,64.4459613,-149.680909,Alaska,E,6,25.2


In [12]:
#Left join score card to yr data
df_yr = pd.merge(df_yr, gun_sc_df, how="left", left_on='state', right_on='State')

In [13]:
df_yr.head(5)

,year_and_quarter,state,value,qtr,Gun Law Strength (Ranked),lat,long,State,Grade,Gun Death Rate (Ranked),Gun Death Rate (per 100K)
0,2020 Q1,Alaska,23.4,2020-03-31,41,64.4459613,-149.680909,Alaska,E,6,25.2
1,2020 Q2,Alaska,23.3,2020-06-30,41,64.4459613,-149.680909,Alaska,E,6,25.2
2,2020 Q3,Alaska,25.3,2020-09-30,41,64.4459613,-149.680909,Alaska,E,6,25.2
3,2020 Q4,Alaska,23.9,2020-12-31,41,64.4459613,-149.680909,Alaska,E,6,25.2
4,2021 Q1,Alaska,23.1,2021-03-31,41,64.4459613,-149.680909,Alaska,E,6,25.2


### Saving final DF as CSV

In [14]:
# Save the data frame to a CSV file
df_yr.to_csv("data/gun_control_results_by_yr.csv")